### Minio 연결 spark session 생성

In [1]:
from minio import Minio
from glob import glob
import os

BUCKET_NAME = "deltalaketest"

client = Minio(
    "localhost:9000",
    access_key = "admin", secret_key = "changeme", secure = False
)

In [2]:
import os

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/jre'

In [3]:
client

In [4]:
buckets = client.list_buckets()

for bucket in buckets:
    print(bucket.name)

deltalaketest
lakehosuetest


In [5]:
client.bucket_exists('deltalaketest')

True

In [6]:
if not client.bucket_exists(BUCKET_NAME):
    client.make_bucket(BUCKET_NAME)

In [7]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# docker-compose.yml 파일 설정한 아이디와 비밀번호
minio_access_key = "admin"
minio_secret_key = "changeme"

# spark session 생성시 aws와 연동하기
def s3_connect_spark(minio_access_key, minio_secret_key):
    # 설정
    conf = (
        SparkConf() # 객체를 사용하여 Spark설정을 정의
        .setAppName("MY_APP") # Spark애플리케이션의 이름을 설정
        .set("spark.hadoop.fs.s3a.access.key", minio_access_key) # Minio 아이디
        .set("spark.hadoop.fs.s3a.secret.key", minio_secret_key) # minio 비번
        .set("spark.hadoop.fs.s3a.endpoint", "http://127.0.0.1:9000") # minio 엔드포인트 설정
        .set("spark.jars.packages", "io.delta:delta-core_2.12:2.1.1, org.apache.hadoop:hadoop-aws:3.3.1") # Delta Lake와 AWS SDK를 Spark에 추가
        .set("spark.databricks.delta.retentionDurationCheck.enabled", "false") # Delta Lake 보존 기간 확인 비활성화
        .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") # Deltalake를 Apache Spark에 확장
        .set("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") # Deltalake를 Apache Spark 카탈로그로 설정
        .set("spark.databricks.delta.properties.defaults.columnMapping.mode","name") # 칼럼 이름에서 공백 및 특수문자 인식
        .set('spark.sql.parquet.columnarReaderBatchSize',100) # Parquet 파일을 읽을 때 사용되는 배치 크기 설정, 한 번에 100개 컬럼을 읽도록 설정
        .set("spark.executor.memory", "8g") # 각 Spark worker의 memory 크기, worker가 사용할 수 있는 메모리 양
        .set("spark.driver.memory", "2g") # Spark Driver의 크기, Driver에 할당된 메모리 양을 지정
    )

    # spark 생성
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
    
    return spark

### 이렇게 하면 연결 성공 했었음

In [8]:
spark = s3_connect_spark(minio_access_key, minio_secret_key)

your 131072x1 screen size is bogus. expect trouble


24/04/08 13:40:56 WARN Utils: Your hostname, Kimbugeon resolves to a loopback address: 127.0.1.1; using 172.22.182.179 instead (on interface eth0)
24/04/08 13:40:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/bugeon/deltalakeenv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/bugeon/.ivy2/cache
The jars for the packages stored in: /home/bugeon/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
 org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2053b745-d9fc-44aa-a344-7a85b2434403;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.1 in central
	found io.delta#delta-storage;2.1.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 2987ms :: artifacts dl 12ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.1 from central in [default]
	io.delta#delta-storage;2.1.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifact

RuntimeError: Java gateway process exited before sending its port number

In [8]:
spark

NameError: name 'spark' is not defined